# Fine-Tuned Medical QA Model 

## Importing Dependencies and Loading Model

In [ ]:
# Initialize tokenizer and model
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-410m")

In [2]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
# Load and preprocess dataset 
from datasets import load_dataset

dataset = load_dataset("Malikeh1375/medical-question-answering-datasets", "all-processed")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', '__index_level_0__'],
        num_rows: 246678
    })
})

In [5]:
dataset=dataset.remove_columns('__index_level_0__')
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 246678
    })
})

In [ ]:
# Split dataset into train and test
split_dataset=dataset['train'].train_test_split(test_size=0.2,shuffle=True,seed=42)
train_dataset=split_dataset['train']
test_dataset=split_dataset['test']

In [7]:
print(train_dataset.shape)
print(test_dataset.shape)

(197342, 3)
(49336, 3)


In [8]:
def tokenize_func(examples):
    # Create structured prompts
    prompts = [
        f"{inst}\n\nPatient: {inp}\n\nDoctor:"
        for inst, inp in zip(examples['instruction'], examples['input'])
    ]
    
    # Combine with responses
    full_texts = [
        p + resp + tokenizer.eos_token
        for p, resp in zip(prompts, examples['output'])
    ]

    # Tokenize with padding/truncation
    tokenized = tokenizer(
        full_texts,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Create labels (mask patient input and instruction)
    tokenized_prompts = tokenizer(
        prompts,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    # Mask everything before the doctor's response
    labels = []
    for i in range(len(tokenized['input_ids'])):
        prompt_len = sum(tokenized_prompts['attention_mask'][i]).item()
        label = [-100]*prompt_len + tokenized['input_ids'][i][prompt_len:].tolist()
        labels.append(label)
    
    tokenized['labels'] = labels
    return tokenized

In [ ]:
# Apply tokenization to datasets  
tokenized_train = train_dataset.map(
    tokenize_func,
    batched=True,
    batch_size=256,
    remove_columns=train_dataset.column_names  # Use the split's columns
)
tokenized_test = test_dataset.map(
    tokenize_func,
    batched=True,
    batch_size=256,
    remove_columns=test_dataset.column_names  # Use the split's columns
)

Map:   0%|          | 0/49336 [00:00<?, ? examples/s]

## Model Training and Evaluation

In [10]:
from transformers import Trainer,TrainingArguments
training_args=TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.1,
    eval_strategy='epoch'
)

In [11]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train.select(range(2000)),
    eval_dataset=tokenized_test.select(range(1000))
)

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.643600,0.589307


TrainOutput(global_step=500, training_loss=0.6435782470703125, metrics={'train_runtime': 8121.3811, 'train_samples_per_second': 0.246, 'train_steps_per_second': 0.062, 'total_flos': 2173886791680000.0, 'train_loss': 0.6435782470703125, 'epoch': 1.0})

In [13]:
trainer.evaluate()

{'eval_loss': 0.5893072485923767,
 'eval_runtime': 1034.2397,
 'eval_samples_per_second': 0.967,
 'eval_steps_per_second': 0.242,
 'epoch': 1.0}

In [26]:
model.save_pretrained("./fine_tuned_ElethuerAI_medical_model")
tokenizer.save_pretrained("./fine_tuned_ElethuerAI_medical_model")

('./fine_tuned_ElethuerAI_medical_model/tokenizer_config.json',
 './fine_tuned_ElethuerAI_medical_model/special_tokens_map.json',
 './fine_tuned_ElethuerAI_medical_model/tokenizer.json')

### Example usage for generating responses:

In [ ]:
import torch
Example usage for generating responses:
prompt = """Act as a doctor.Keep annotations concise. 
Question: "I’ve had sharp pain in my upper abdomen after eating."
Answer:"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    inputs.input_ids,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

response = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
print("Generated:", response)

Generated: This may be related to a rupture of the gallbladder.  The pain may be related to the rupture of the gallbladder.  This may be related to a gallbladder stone.  This may be related to a gallbladder infection.  This may be related to an intestines infection.  This may be related to a gallbladder infection.  This may be related to a gallbladder stone.  This may be related to a gallbladder infection.
